# 3.0 - Modelado, ajuste y evaluacion

In [ ]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

from catboost import CatBoostRegressor as CTR

from sklearn.model_selection import train_test_split as tts 

from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2